In [5]:
import pandas as pd
import os 

os.chdir('..')
articles = pd.read_csv('Data/articles_with_scores.csv')

In [7]:
articles["date"] = pd.to_datetime(dict(
    year=articles["annee"],
    month=articles["mois"],
    day=articles["jour"]
))

articles["post_metoo"] = articles["date"] >= "2017-10-01"


In [8]:
articles

,Unnamed: 0,identifiant,journal_clean,titre,annee,mois,jour,texte_total,score_final,voc_violence_final,hierar_final,portrait_victime_final,portrait_auteur_final,relation_final,meanisme_violence_final,stat_final,mean_variables,date,post_metoo
0,0,3a4723d9d754ca30c68ba8e420cd6683548f2af7ff7af3...,Libération,Viol collectif dans la nuit de la Saint-Sylves...,1995,1,6,Viol collectif dans la nuit de la Saint-Sylves...,0.620987,1.0,1.0,1.0,1.0,1.0,0.0,-0.0,0.714286,1995-01-06,False
1,1,2b6ce23c0fbfd6e213b7cd196a4b12eda4323e322f5f35...,Libération,"En Moselle, une jeune femme tuée par son mari",1995,1,16,"En Moselle, une jeune femme tuée par son mari ...",0.195183,0.0,1.0,0.0,0.0,1.0,-0.0,-1.0,0.142857,1995-01-16,False
2,2,2293f5c041bec1dcdfe52bbe2655e0bec9a449310f534b...,Libération,Narbonne: une femme tuée après une fusillade. ...,1995,4,11,Narbonne: une femme tuée après une fusillade. ...,0.391785,0.0,1.0,1.0,1.0,1.0,0.0,-0.0,0.571429,1995-04-11,False
3,3,2e9c275968c7996e9407977dfb6aaacd7141ac5e7ed0a2...,Libération,Une femme enceinte agressée par des vigiles au...,1995,4,17,Une femme enceinte agressée par des vigiles au...,0.476032,1.0,1.0,1.0,1.0,1.0,0.0,-0.0,0.714286,1995-04-17,False
4,4,b1987151ea7e9e657b7207398a434df106c3a9d8d3c709...,Libération,Une femme blesse son ex-ami avant de se suicider,1995,9,18,Une femme blesse son ex-ami avant de se suicid...,-0.051603,0.0,1.0,0.0,0.0,0.0,-0.0,-1.0,0.000000,1995-09-18,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4477,4477,a3c235da28477a6045b1e97abd2b5491b1aa772c273ae4...,Libération,Me Too : Harvey Weinstein hospitalisé après de...,2024,12,3,Me Too : Harvey Weinstein hospitalisé après de...,0.708056,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.714286,2024-12-03,True
4478,4478,377988b4b18f66bdf316259434127bc49e39ac016edacd...,Le Figaro,Italie: un homme condamné à la prison à vie po...,2024,12,3,Italie: un homme condamné à la prison à vie po...,0.595197,1.0,0.0,1.0,1.0,1.0,0.0,-0.0,0.571429,2024-12-03,True
4479,4479,1a7689437dc0d5a11e37df86ddd7d9ea4c37195a7ecdbc...,Le Point,Harvey Weinstein est de nouveau hospitalisé,2024,12,4,Harvey Weinstein est de nouveau hospitalisé Ha...,0.721072,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.857143,2024-12-04,True
4480,4480,20d7c133c259fe5e149d1cddfa8be63aa616233362596f...,Le Figaro,«Adèle Haenel et ses 12 ans étaient d’une sens...,2024,12,4,«Adèle Haenel et ses 12 ans étaient d’une sens...,0.781516,1.0,0.0,1.0,1.0,1.0,1.0,-0.0,0.714286,2024-12-04,True


In [11]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
label_cols = [
    "voc_violence", "hierar", "portrait_victime",
    "portrait_auteur", "relation", "meanisme_violence", "stat"
]

# 1. Création des groupes avant/après MeToo
pre = articles[articles["post_metoo"] == False]
post = articles[articles["post_metoo"] == True]

# 2. Variables à comparer
vars_to_compare = ["mean_variables"] + [col + "_final" for col in label_cols]

# 3. Comparaison des moyennes et t-test
results = []

for var in vars_to_compare:
    pre_vals = pre[var].dropna()
    post_vals = post[var].dropna()
    
    mean_pre = pre_vals.mean()
    mean_post = post_vals.mean()
    diff = mean_post - mean_pre
    
    tstat, pval = ttest_ind(post_vals, pre_vals, equal_var=False)
    
    results.append({
        "variable": var,
        "mean_pre": round(mean_pre, 3),
        "mean_post": round(mean_post, 3),
        "diff": round(diff, 3),
        "p_value": round(pval, 4)
    })

df_results = pd.DataFrame(results)


In [12]:
df_results

,variable,mean_pre,mean_post,diff,p_value
0,mean_variables,0.383,0.648,0.265,0.0
1,voc_violence_final,0.472,0.870,0.398,0.0
2,hierar_final,0.602,0.411,-0.191,0.0
3,portrait_victime_final,0.583,0.886,0.303,0.0
4,portrait_auteur_final,0.669,0.900,0.231,0.0
5,relation_final,0.525,0.884,0.359,0.0
6,meanisme_violence_final,0.128,0.628,0.500,0.0
7,stat_final,-0.302,-0.045,0.257,0.0
